### Import Libraries


In [ ]:
%pip install mediapipe

In [ ]:
import os
import cv2
import time
import numpy as np
import pandas as pd
import mediapipe as mp
from tqdm import tqdm


### Output Video Creation 

In [ ]:
def create_output_video(face_frames, output_path, fps, max_frames):
    writer = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (112, 112))
    for i in range(max_frames):
        writer.write(face_frames[i] if i < len(face_frames) else np.zeros((112, 112, 3), dtype=np.uint8))
    writer.release()


### Video Processing 

In [ ]:
def process_video(video_path, output_folder, detector, max_frames=300):
    cap = cv2.VideoCapture(video_path)
    fps, width, height = cap.get(cv2.CAP_PROP_FPS), int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    output_path = os.path.join(output_folder, f"{os.path.splitext(os.path.basename(video_path))[0]}.mp4")
    os.makedirs(output_folder, exist_ok=True)

    face_frames = []
    while True:
        ret, frame = cap.read()
        if not ret: break

        results = detector.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        if results.detections:
            ih, iw, _ = frame.shape
            for detection in results.detections:
                bbox = detection.location_data.relative_bounding_box
                x1, y1, x2, y2 = [int(max(min(v * dim, dim - 1), 0)) for v, dim in zip(
                    [bbox.xmin, bbox.ymin, bbox.xmin + bbox.width, bbox.ymin + bbox.height], [iw, ih, iw, ih])]
                face = frame[y1:y2, x1:x2]
                if face.size > 0:
                    face_frames.append(cv2.resize(face, (112, 112)))

    cap.release()
    create_output_video(face_frames, output_path, fps, max_frames)
    return output_path


### Folder Processing 

In [ ]:
def process_folder(input_folder, output_folder):
    detector = mp.solutions.face_detection.FaceDetection(min_detection_confidence=0.7)
    video_files = [f for f in os.listdir(input_folder) if f.endswith(('.mp4', '.avi', '.mov'))]
    all_face_videos = []

    with tqdm(total=len(video_files), desc="Overall Progress") as pbar:
        for video_file in video_files:
            video_path = os.path.join(input_folder, video_file)
            all_face_videos.append(process_video(video_path, output_folder, detector))
            pbar.update(1)

    detector.close()
    return all_face_videos


### Main Function

In [ ]:
def main():
    input_folder = "path/to/input/folder"
    output_folder = "path/to/output/folder"
    
    start_time = time.time()
    face_videos = process_folder(input_folder, output_folder)
    total_time = time.time() - start_time

    total_videos = len([f for f in os.listdir(input_folder) if f.endswith(('.mp4', '.avi', '.mov'))])
    print(f"Detected faces: {len(face_videos)}")
    print(f"Total time taken: {total_time:.2f} seconds")
    print(f"Total videos processed: {total_videos}")

if __name__ == "__main__":
    main()
